# Batch Analysis for hawk/dove simulation with multiple risk attitudes, no adjustment

- What % of time do agents play Hawk, by risk attitude? risk-inclined (R=1) and risk-avoidant (R8) agents play Hawk?
- Cumulative wealth analysis by risk attitude

To track how often agents play Hawk, we need to collect data for every round.

This data was generated by running:
```console
simulatingrisk/hawkdovemulti/batch_run.py --params no_adjustment --agent-data --collect-data every_round
```

Each row in the data file represents one round for each agent.

In [172]:
import polars as pl


# df = pd.read_csv("../../data/hawkdovemulti/2025-07-22T170057_747737_agent.csv")
# df = pd.read_csv("../../data/hawkdovemulti/2025-07-23T135510_859267_agent.csv")
# which batch run data to use; use variable to ensure we use matching agent and model data
batch_run_date = "2025-07-24T120337_924060"

# load agent data, drop unneeded columns, and add numeric value 1 for played hawk, 0 for played dove

df = (
    pl.read_csv(f"../../data/hawkdovemulti/{batch_run_date}_agent.csv")
        .drop("risk_level_changed")  # drop risk_level_changed; not relevant here (no adjustment = no changes)
        .rename({'risk_level': 'risk_attitude'})  # code still uses risk_level internally; relabel as risk attitude
        .with_columns(
            # add a numeric field to turn choice of play to 1/0 hawk, for aggregation            
            played_hawk=pl.when(pl.col("choice").eq("hawk")).then(1).otherwise(0)
        )
)
df.head()

RunId,iteration,Step,AgentID,risk_attitude,choice,points,played_hawk
i64,i64,i64,i64,i64,str,i64,i32
1,1,1,0,3,"""dove""",12,0
1,1,1,1,8,"""dove""",12,0
1,1,1,2,8,"""hawk""",12,1
1,1,1,3,4,"""hawk""",18,1
1,1,1,4,8,"""hawk""",15,1


## Percent of the time agents play Hawk, by risk attitude

What % of time do risk-inclined (R=1) and risk-avoidant (R8) agents play Hawk?

- Guess from observation is is >90% for R=1, <10% for R8, but we want to have statistics for this: for X trials, how many of them does R1 play Hawk more than 90% of the time?
- Also useful to have statistics e.g. R=2 played Hawk between 80-90% of the time, or whatever the result is.


In [208]:
# each row in the data frame is a play by an agent on the grid
# group by risk level, then:
# - count the number of rows (= total number of plays)
# - sum the played_hawk field (= number of times played hawk)
# - calculate percent of turns played hawk

hawk_by_risk_attitude = (
    df.group_by("risk_attitude")
        .agg(n_plays=pl.col("played_hawk").count(), n_plays_hawk=pl.col("played_hawk").sum())
    .with_columns(
        pct_plays_hawk=pl.col("n_plays_hawk").truediv(pl.col("n_plays")).mul(100)
    )
)

In [209]:
# output core fields as nicely styled table
(hawk_by_risk_attitude
    .select("risk_attitude", "pct_plays_hawk")
    .sort("risk_attitude")
    .rename({"risk_attitude": "Risk Attitude", "pct_plays_hawk": "% plays Hawk"})
    .style.tab_header(title="% of time agents play Hawk by Risk Attitude")
    .fmt_number("% plays Hawk", decimals=1)
)

GT(_tbl_data=shape: (10, 2)
┌───────────────┬──────────────┐
│ Risk Attitude ┆ % plays Hawk │
│ ---           ┆ ---          │
│ i64           ┆ f64          │
╞═══════════════╪══════════════╡
│ 0             ┆ 98.649774    │
│ 1             ┆ 93.023643    │
│ 2             ┆ 86.810279    │
│ 3             ┆ 70.359122    │
│ 4             ┆ 60.247899    │
│ 5             ┆ 39.645375    │
│ 6             ┆ 29.6279      │
│ 7             ┆ 13.210993    │
│ 8             ┆ 6.951191     │
│ 9             ┆ 1.355157     │
└───────────────┴──────────────┘, _body=<great_tables._gt_data.Body object at 0x338a4da60>, _boxhead=Boxhead([ColInfo(var='Risk Attitude', type=<ColInfoTypeEnum.default: 1>, column_label='Risk Attitude', column_align='right', column_width=None), ColInfo(var='% plays Hawk', type=<ColInfoTypeEnum.default: 1>, column_label='% plays Hawk', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x3d4ec7020>, _spanners=Spanners([]), _heading=Heading(title='% of time agents play Hawk by Risk Attitude', subtitle=None, preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x3d4ec4800>, _formats=[<great_tables._gt_data.FormatInfo object at 0x3d4ec57c0>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_right_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_right_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_right_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), table_border_bottom_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_bottom_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_bottom_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_bottom_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_left_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_left_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_left_color=OptionsInfo(scss=True, category='table', type='value', v

In [175]:
import altair as alt

alt.Chart(hawk_by_risk_attitude).mark_bar(width=10).encode(
    x=alt.X("risk_attitude", title="Risk Attitude").scale(domain=[0, 9]),                                                          
    y=alt.Y("pct_plays_hawk", title="% time plays Hawk")
).properties(title="% of time agents play Hawk")

alt.Chart(...)

Context: how many runs are these numbers drawn from?



In [176]:
total_unique_runs = len(df["RunId"].unique())
total_iterations = len(df["iteration"].unique())
n_combinations = int(total_unique_runs / total_iterations)

# Step is the round count indicator
longest_run = df["Step"].max()    # highest across all runs
# average of max value for each run. Group by run, get max Step, average —> returns a dataframe, get first Step value
average_run = df.group_by("RunId").agg(pl.col("Step").max()).mean()["Step"].first()

print(f"""{total_unique_runs:,} total unique runs; {total_iterations} iterations of {n_combinations} different parameter combinations.

Longest run: {longest_run} steps
Average run: {average_run:.1f} steps
""")


13,500 total unique runs; 100 iterations of 135 different parameter combinations.

Longest run: 109 steps
Average run: 44.9 steps



### Analysis filterd by simulation parameters

In [177]:
# identify the last round of each run
# for both wealth analysis and model parameters, we want to look at the last round (Step) of each run
last_round_df = df.group_by("RunId").agg(pl.col("Step").max())


In [178]:
# load model data and filter to last round for each run
full_model_df = pl.read_csv(f"../../data/hawkdovemulti/{batch_run_date}_model.csv")
model_df = last_round_df.join(full_model_df, on=['RunId', 'Step'], how="left")
# limit to only those fields that are needed for our analysis
model_df = model_df.select("RunId", "iteration", "risk_distribution", "play_neighborhood", "observed_neighborhood", "grid_size")
model_df.head()

RunId,iteration,risk_distribution,play_neighborhood,observed_neighborhood,grid_size
i64,i64,str,i64,i64,i64
9054,54,"""skewed right""",24,8,5
11427,27,"""bimodal""",8,4,5
7887,87,"""skewed left""",4,4,5
2361,61,"""uniform""",4,24,25
1313,13,"""uniform""",24,24,10


In [179]:
print(f"""Simulation parameters:

Grid size: {', '.join(str(n) for n in sorted(model_df["grid_size"].unique()))}
Iniital risk distribution: {', '.join(str(n) for n in sorted(model_df["risk_distribution"].unique()))}
Play neighborhood size: {', '.join(str(n) for n in sorted(model_df["play_neighborhood"].unique()))}
Observed neighborhood sized: {', '.join(str(n) for n in sorted(model_df["observed_neighborhood"].unique()))}

""")

Simulation parameters:

Grid size: 5, 10, 25
Iniital risk distribution: bimodal, normal, skewed left, skewed right, uniform
Play neighborhood size: 4, 8, 24
Observed neighborhood sized: 4, 8, 24




In [180]:
# join agent data with model data so we can filter by starting parameters
agent_df_params = df.join(model_df, on=["RunId", "iteration"], how="left")

#### Grid size


In [181]:
hawk_by_gridsize_riskattitude = (
    agent_df_params.group_by("grid_size", "risk_attitude")
         .agg(n_plays=pl.col("played_hawk").count(), n_plays_hawk=pl.col("played_hawk").sum())
    .with_columns(
        pct_plays_hawk=pl.col("n_plays_hawk").truediv(pl.col("n_plays")).mul(100)
    )
)

alt.Chart(hawk_by_gridsize_riskattitude).mark_bar(width=10).encode(
    x=alt.X("risk_attitude", title="Risk Attitude").scale(domain=[0, 9]),                                                          
    y=alt.Y("pct_plays_hawk", title="% time plays Hawk")
).facet(column=alt.Column("grid_size", title="Grid Size")).properties(title="% of time agents play Hawk")


alt.FacetChart(...)

#### Play neighborhood


In [182]:
hawk_by_playnhood_riskattitude = (
    agent_df_params.group_by("play_neighborhood", "risk_attitude")
         .agg(n_plays=pl.col("played_hawk").count(), n_plays_hawk=pl.col("played_hawk").sum())
    .with_columns(
        pct_plays_hawk=pl.col("n_plays_hawk").truediv(pl.col("n_plays")).mul(100)
    )
)

alt.Chart(hawk_by_playnhood_riskattitude).mark_bar(width=10).encode(
    x=alt.X("risk_attitude", title="Risk Attitude").scale(domain=[0, 9]),                                                          
    y=alt.Y("pct_plays_hawk", title="% time plays Hawk")
).facet(column=alt.Column("play_neighborhood", title="Play Neighborhood")).properties(title="% of time agents play Hawk")


alt.FacetChart(...)

#### Observed neighborhood


In [183]:
hawk_by_obsnhood_riskattitude = (
    agent_df_params.group_by("observed_neighborhood", "risk_attitude")
         .agg(n_plays=pl.col("played_hawk").count(), n_plays_hawk=pl.col("played_hawk").sum())
    .with_columns(
        pct_plays_hawk=pl.col("n_plays_hawk").truediv(pl.col("n_plays")).mul(100)
    )
)

alt.Chart(hawk_by_obsnhood_riskattitude).mark_bar(width=10).encode(
    x=alt.X("risk_attitude", title="Risk Attitude").scale(domain=[0, 9]),                                                          
    y=alt.Y("pct_plays_hawk", title="% time plays Hawk")
).facet(column=alt.Column("observed_neighborhood", title="Observed Neighborhood")).properties(title="% of time agents play Hawk")


alt.FacetChart(...)

#### Initial risk distribution


In [184]:
hawk_by_initialdist_riskattitude = (
    agent_df_params.group_by("risk_distribution", "risk_attitude")
         .agg(n_plays=pl.col("played_hawk").count(), n_plays_hawk=pl.col("played_hawk").sum())
    .with_columns(
        pct_plays_hawk=pl.col("n_plays_hawk").truediv(pl.col("n_plays")).mul(100)
    )
)

alt.Chart(hawk_by_initialdist_riskattitude).mark_bar(width=10).encode(
    x=alt.X("risk_attitude", title="Risk Attitude").scale(domain=[0, 9]),                                                          
    y=alt.Y("pct_plays_hawk", title="% time plays Hawk")
).facet(facet=alt.Facet("risk_distribution", title="Initial Risk Distribution"), columns=3
).properties(title="% of time agents play Hawk")


alt.FacetChart(...)

## Cumulative Wealth analysis


- mean and quartiles for wealth by R
- does mean vary between Rs or is it roughly the same?
  - quartiles look different, but need a statistic; esp. compare lower-R quartile to higher-R quartile
     - expect/hope that lower quartile is higher for R1 than R8, higher quartile is higher for R8 than R1
     - what's going on in the middle?


-----

Because our simulations run for different lengths before stopping, and because our simulations include a range of different play neighborhoods (which affects total payoff), we scale points (wealth) by play neighborhood and simulation run length for comparison across simulations.  The scaled points is multipled by 100 to make the point ranges more intelligible and comparable to a simulation run.

$ scaled\_points = ((points / play\_neighborhood) / simulation\_runlength) * 100 $


In [210]:
# combine the last round dataframe and full agent dataframe to get just the last round of each run
agents_last_round_df = (
    last_round_df.join(df, on=['RunId', 'Step'], how="left")
        # join on model parameters, for filtering and scaling by play neighborhood
        .join(model_df, on=["RunId", "iteration"])
        .with_columns(
            # calculate a scaled points value so we can compare across runs with different length and play neighborhood
            scaled_points=pl.col("points").truediv(pl.col("play_neighborhood")).truediv(pl.col("Step")).mul(100)
        )
    
)

Now plot scaled wealth distribution by risk attitude.

In [186]:
wealthchart_title = alt.TitleParams(
    "Cumulative Wealth by Risk Attitude",
    subtitle=["Wealth scaled by simulation length and play neighborhood"]
)

cum_wealth_boxplot = alt.Chart(agents_last_round_df, title=wealthchart_title).mark_boxplot(extent='min-max').encode(
    x=alt.X("risk_attitude", title="Risk Attitude").scale(domain=[0, 9]),
    y=alt.X("scaled_points", title="Wealth "),
) 
cum_wealth_boxplot

alt.Chart(...)

To check our scaling, we compared it with cumulative wealth at round 31 across all simulations, scaled only by play neighborhood; the distribution was similar to the scaled wealth plot.

Calculate quartiles and other statistics and output as a table, for reference.

In [187]:
# Altair boxplot is calculating quartiles for us, but we can calculate them directly as well
wealth_by_risk_attitude = agents_last_round_df.group_by("risk_attitude").agg(
    min=pl.col("scaled_points").min(), 
    max=pl.col("scaled_points").max(),
    median=pl.col("scaled_points").median(), 
    mean=pl.col("scaled_points").mean(), 
    Q1=pl.col("scaled_points").quantile(0.25),
    Q2=pl.col("scaled_points").quantile(0.5),
    Q3=pl.col("scaled_points").quantile(0.75),
).sort("risk_attitude")

In [213]:
# define a custom box plot method using layered plots,
# so that we can quickly generate plots from statistics generated by polars

def custom_boxplot(df):        
    # calculate risk group labels, for color
    plot_df = df.with_columns(
        risk_type=pl.when(pl.col.risk_attitude < 3).then(pl.lit("Risk Inclined")).when(pl.col.risk_attitude < 7).then(pl.lit("Risk Moderate")).otherwise(pl.lit("Risk Avoidant"))
    )
    # create base chart to use across layers
    base_chart = alt.Chart(plot_df)
    
    # area chart for Q1 to Q3
    area_chart = base_chart.mark_rect(width=15).encode(
        y=alt.Y('Q1').axis(offset=12),  # add offset so axis does not crowd rectangle
        y2='Q3', 
        x=alt.X('risk_attitude:Q', title='Risk Attitude'), 
        tooltip=['min', 'max', 'mean', 'Q1', 'median', 'Q3'],
        color=alt.Color('risk_type', title='Risk Attitude')
    )
    # line chart for min-max spread
    # specifying a stroke for point on the line only adds the min point
    minmax_line_chart = base_chart.mark_line(
        point=alt.OverlayMarkDef(filled=False, shape='stroke', color='black', strokeWidth=2)
    ).encode(
        alt.Y('min'),
        alt.Y2('max'),
        x='risk_attitude:Q'
    )
    # add a black stroke for the max
    max_marks = base_chart.mark_point(shape='stroke', size=55, color='black').encode(
        y='max',
        x=alt.X('risk_attitude:Q'),
    )
    # add a black stroke for the min
    median_marks = base_chart.mark_point(shape='stroke', size=100, strokeWidth=1, color='white').encode(
        y='median',
        x='risk_attitude:Q')

    # mean line ? 
    mean_line_chart = base_chart.mark_line(interpolate="monotone", color="black", opacity=0.5).encode(
        x=alt.X("risk_attitude:Q"),
        y=alt.Y("mean", title="Wealth (mean)").scale(zero=False),
    )
    
    return (mean_line_chart + minmax_line_chart + area_chart + median_marks + max_marks).resolve_axis('shared')


This is a customized box plot, which includes all the same information as the simpler version above. This one is based on numbers pre-calculated on the polars dataframe, which is more efficient than altair. 

This version adds strokes for the min and max values (whiskers), colors the Q1-Q3 box portion of the plot based on the risk attitude groups, and adds a line graph of the mean behind the box plot.

Mouse over the box to view statistics for each risk attitude (min, max, mean, Q1-Q3).

In [214]:
custom_boxplot(wealth_by_risk_attitude).properties(title=wealthchart_title)

alt.LayerChart(...)

In [190]:
# output wealth by risk attitude as nicely styled table

(wealth_by_risk_attitude
    .rename({"risk_attitude": "Risk Attitude"})
    .style.tab_header(title="Cumulative Wealth by Risk Attitude", subtitle="Wealth scaled by simulation length and play neighborhood")
    .fmt_number(decimals=1)
    .fmt_number("Risk Attitude", decimals=0)
)

GT(_tbl_data=shape: (10, 8)
┌────────────┬───────────┬────────────┬────────────┬───────────┬───────────┬───────────┬───────────┐
│ Risk       ┆ min       ┆ max        ┆ median     ┆ mean      ┆ Q1        ┆ Q2        ┆ Q3        │
│ Attitude   ┆ ---       ┆ ---        ┆ ---        ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
│ ---        ┆ f64       ┆ f64        ┆ f64        ┆ f64       ┆ f64       ┆ f64       ┆ f64       │
│ i64        ┆           ┆            ┆            ┆           ┆           ┆           ┆           │
╞════════════╪═══════════╪════════════╪════════════╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 0          ┆ 0.0       ┆ 300.0      ┆ 149.691358 ┆ 148.68525 ┆ 114.91935 ┆ 149.69135 ┆ 174.59677 │
│            ┆           ┆            ┆            ┆ 3         ┆ 5         ┆ 8         ┆ 4         │
│ 1          ┆ 0.0       ┆ 300.0      ┆ 147.580645 ┆ 144.02010 ┆ 113.20621 ┆ 147.58064 ┆ 167.79661 │
│            ┆           ┆            ┆            ┆ 5         ┆ 5         ┆ 5         ┆           │
│ 2          ┆ 0.0       ┆ 300.0      ┆ 147.580645 ┆ 145.42954 ┆ 112.5     ┆ 147.58064 ┆ 170.56451 │
│            ┆           ┆            ┆            ┆ 5         ┆           ┆ 5         ┆ 6         │
│ 3          ┆ 0.0       ┆ 300.0      ┆ 137.5      ┆ 143.42071 ┆ 110.84905 ┆ 137.5     ┆ 165.95744 │
│            ┆           ┆            ┆            ┆ 2         ┆ 7         ┆           ┆ 7         │
│ 4          ┆ 0.0       ┆ 300.0      ┆ 134.659091 ┆ 141.71522 ┆ 109.94623 ┆ 134.65909 ┆ 157.25806 │
│            ┆           ┆            ┆            ┆ 5         ┆ 7         ┆ 1         ┆ 5         │
│ 5          ┆ 1.694915  ┆ 300.0      ┆ 137.5      ┆ 139.42156 ┆ 114.34426 ┆ 137.5     ┆ 150.80645 │
│            ┆           ┆            ┆            ┆           ┆ 2         ┆           ┆ 2         │
│ 6          ┆ 43.220339 ┆ 300.0      ┆ 143.025362 ┆ 143.34745 ┆ 125.40322 ┆ 143.02536 ┆ 154.30107 │
│            ┆           ┆            ┆            ┆ 5         ┆ 6         ┆ 2         ┆ 5         │
│ 7          ┆ 64.705882 ┆ 300.0      ┆ 149.327957 ┆ 147.04836 ┆ 137.03703 ┆ 149.32795 ┆ 158.06451 │
│            ┆           ┆            ┆            ┆ 9         ┆ 7         ┆ 7         ┆ 6         │
│ 8          ┆ 87.096774 ┆ 300.0      ┆ 150.403226 ┆ 151.04103 ┆ 140.25423 ┆ 150.40322 ┆ 161.69354 │
│            ┆           ┆            ┆            ┆ 6         ┆ 7         ┆ 6         ┆ 8         │
│ 9          ┆ 96.774194 ┆ 203.225806 ┆ 150.0      ┆ 150.28736 ┆ 141.39784 ┆ 150.0     ┆ 160.88709 │
│            ┆           ┆            ┆            ┆ 5         ┆ 9         ┆           ┆ 7         │
└────────────┴───────────┴────────────┴────────────┴───────────┴───────────┴───────────┴───────────┘, _body=<great_tables._gt_data.Body object at 0x337d554f0>, _boxhead=Boxhead([ColInfo(var='Risk Attitude', type=<ColInfoTypeEnum.default: 1>, column_label='Risk Attitude', column_align='right', column_width=None), ColInfo(var='min', type=<ColInfoTypeEnum.default: 1>, column_label='min', column_align='right', column_width=None), ColInfo(var='max', type=<ColInfoTypeEnum.default: 1>, column_label='max', column_align='right', column_width=None), ColInfo(var='median', type=<ColInfoTypeEnum.default: 1>, column_label='median', column_align='right', column_width=None), ColInfo(var='mean', type=<ColInfoTypeEnum.default: 1>, column_label='mean', column_align='right', column_width=None), ColInfo(var='Q1', type=<ColInfoTypeEnum.default: 1>, column_label='Q1', column_align='right', column_width=None), ColInfo(var='Q2', type=<ColInfoTypeEnum.default: 1>, column_label='Q2', column_align='right', column_width=None), ColInfo(var='Q3', type=<ColInfoTypeEnum.default: 1>, column_label='Q3', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x3d4ead0d0>, _spanners=Spanners([]), _heading=Heading(title='Cumulative Wealth by Risk Attitude', subtitle='Wealth scaled by simulation length and play neighborhood', preh

In [191]:
# define a custom method to plot mean and quartiles for a specified simulation parameter

def plot_mean_quartiles(df, field, field_label):
    # takes a dataframe, the field in the dataframe for the simulation parameter, and displayable label for the field
    
    # create a selection filter bound to the legend
    selection = alt.selection_point(fields=[field], bind='legend')
    base_chart = alt.Chart(df)
    # curved line for the mean wealth by risk attitude
    wealth_mean_chart = base_chart.mark_line(interpolate="monotone").encode(
        x=alt.X("risk_attitude", title="Risk Attitude").scale(domain=[0, 9]),
        y=alt.Y("mean", title="Wealth (mean)").scale(zero=False),
        color=alt.Color(f"{field}:N", title=field_label),
        opacity=alt.when(selection).then(alt.value(1.0)).otherwise(alt.value(0.4))
    ).add_params(
        selection
    )
    # curved area chart for wealth quartile spread
    wealth_spread = base_chart.mark_area(interpolate="monotone").encode(
        x=alt.X("risk_attitude", title="Risk Attitude").scale(domain=[0, 9]),
        y=alt.Y("Q3").scale(zero=False),
        y2="Q1",
        color=alt.Color(f"{field}:N", title=field_label),
        opacity=alt.when(selection).then(alt.value(0.3)).otherwise(alt.value(0.1))
    ).add_params(
        selection
    )
    
    # combine the charts for multiple ways to view
    chart_wealth_title = wealthchart_title.copy()
    chart_wealth_title['text'] = f"{wealthchart_title['text']} and {field_label} — Mean and Quartiles"

    # display mean curve chart, wealth area chart, and mean layered with the wealth
    return (wealth_mean_chart | wealth_spread | (wealth_mean_chart + wealth_spread)).resolve_legend(color="shared").properties(title=chart_wealth_title)


### Analysis filtered by simulation parameters

How does the wealth distribution vary based on other starting parameters?

#### Grid size

In [192]:
# calculate mean & quartiles for risk attitude by grid size
wealth_by_risk_grid = agents_last_round_df.group_by("risk_attitude", "grid_size").agg(
    min=pl.col("scaled_points").min(), 
    max=pl.col("scaled_points").max(),
    median=pl.col("scaled_points").median(),
    mean=pl.col("scaled_points").mean(), 
    Q1=pl.col("scaled_points").quantile(0.25),
    Q2=pl.col("scaled_points").quantile(0.5),
    Q3=pl.col("scaled_points").quantile(0.75),
).sort("grid_size", "risk_attitude")


In [215]:
custom_boxplot(wealth_by_risk_grid).facet(column=alt.Column("grid_size", title="Grid Size")).properties(title=wealthchart_title)

alt.FacetChart(...)

In [194]:
plot_mean_quartiles(wealth_by_risk_grid, 'grid_size', 'Grid Size')

alt.HConcatChart(...)

Click on items in the Grid Size legend to change the chart opacity to focus on a particular group.

In [195]:
# turn grid size into label for context in the table
(wealth_by_risk_grid.with_columns(grid_size=pl.lit("Grid Size: ").add(pl.col("grid_size").cast(pl.datatypes.String)))
    .rename({"risk_attitude": "Risk Attitude"})
    .style.tab_header(title="Cumulative Wealth by Risk Attitude and Grid Size", subtitle="Wealth scaled by simulation length and play neighborhood")
     .tab_stub(rowname_col="Risk Attitude", groupname_col="grid_size")
    .fmt_number(decimals=1)
    .fmt_number("Risk Attitude", decimals=0)
)

GT(_tbl_data=shape: (30, 9)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ Risk      ┆ grid_size ┆ min       ┆ max       ┆ … ┆ mean      ┆ Q1        ┆ Q2        ┆ Q3       │
│ Attitude  ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│ ---       ┆ str       ┆ f64       ┆ f64       ┆   ┆ f64       ┆ f64       ┆ f64       ┆ f64      │
│ i64       ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 0         ┆ Grid      ┆ 0.0       ┆ 300.0     ┆ … ┆ 148.22116 ┆ 125.0     ┆ 150.0     ┆ 161.9623 │
│           ┆ Size: 5   ┆           ┆           ┆   ┆ 5         ┆           ┆           ┆ 66       │
│ 1         ┆ Grid      ┆ 0.0       ┆ 300.0     ┆ … ┆ 142.61666 ┆ 113.15789 ┆ 148.61111 ┆ 161.5591 │
│           ┆ Size: 5   ┆           ┆           ┆   ┆           ┆ 5         ┆ 1         ┆ 4        │
│ 2         ┆ Grid      ┆ 0.0       ┆ 300.0     ┆ … ┆ 143.50073 ┆ 109.44444 ┆ 147.61904 ┆ 167.8672 │
│           ┆ Size: 5   ┆           ┆           ┆   ┆ 9         ┆ 4         ┆ 8         ┆ 32       │
│ 3         ┆ Grid      ┆ 1.415094  ┆ 300.0     ┆ … ┆ 140.98151 ┆ 104.43548 ┆ 136.96236 ┆ 162.0967 │
│           ┆ Size: 5   ┆           ┆           ┆   ┆ 2         ┆ 4         ┆ 6         ┆ 74       │
│ 4         ┆ Grid      ┆ 2.419355  ┆ 300.0     ┆ … ┆ 139.74679 ┆ 103.22580 ┆ 133.02238 ┆ 156.8502 │
│           ┆ Size: 5   ┆           ┆           ┆   ┆ 4         ┆ 6         ┆ 8         ┆ 82       │
│ …         ┆ …         ┆ …         ┆ …         ┆ … ┆ …         ┆ …         ┆ …         ┆ …        │
│ 5         ┆ Grid      ┆ 1.694915  ┆ 300.0     ┆ … ┆ 139.60384 ┆ 115.10416 ┆ 137.5     ┆ 150.8064 │
│           ┆ Size: 25  ┆           ┆           ┆   ┆           ┆ 7         ┆           ┆ 52       │
│ 6         ┆ Grid      ┆ 43.220339 ┆ 300.0     ┆ … ┆ 143.56589 ┆ 125.45180 ┆ 143.14516 ┆ 154.4354 │
│           ┆ Size: 25  ┆           ┆           ┆   ┆ 8         ┆ 7         ┆ 1         ┆ 84       │
│ 7         ┆ Grid      ┆ 64.705882 ┆ 300.0     ┆ … ┆ 147.19194 ┆ 137.09677 ┆ 149.38524 ┆ 158.0645 │
│           ┆ Size: 25  ┆           ┆           ┆   ┆ 5         ┆ 4         ┆ 6         ┆ 16       │
│ 8         ┆ Grid      ┆ 87.096774 ┆ 300.0     ┆ … ┆ 151.13959 ┆ 140.32258 ┆ 150.42372 ┆ 161.6935 │
│           ┆ Size: 25  ┆           ┆           ┆   ┆ 5         ┆ 1         ┆ 9         ┆ 48       │
│ 9         ┆ Grid      ┆ 96.774194 ┆ 203.22580 ┆ … ┆ 150.37525 ┆ 141.26344 ┆ 150.06944 ┆ 160.8870 │
│           ┆ Size: 25  ┆           ┆ 6         ┆   ┆ 2         ┆ 1         ┆ 4         ┆ 97       │
└───────────┴───────────┴───────────┴───────────┴───┴───────────┴───────────┴───────────┴──────────┘, _body=<great_tables._gt_data.Body object at 0x3d4eb0650>, _boxhead=Boxhead([ColInfo(var='Risk Attitude', type=<ColInfoTypeEnum.stub: 2>, column_label='Risk Attitude', column_align='right', column_width=None), ColInfo(var='grid_size', type=<ColInfoTypeEnum.row_group: 3>, column_label='grid_size', column_align='left', column_width=None), ColInfo(var='min', type=<ColInfoTypeEnum.default: 1>, column_label='min', column_align='right', column_width=None), ColInfo(var='max', type=<ColInfoTypeEnum.default: 1>, column_label='max', column_align='right', column_width=None), ColInfo(var='median', type=<ColInfoTypeEnum.default: 1>, column_label='median', column_align='right', column_width=None), ColInfo(var='mean', type=<ColInfoTypeEnum.default: 1>, column_label='mean', column_align='right', column_width=None), ColInfo(var='Q1', type=<ColInfoTypeEnum.default: 1>, column_label='Q1', column_align='right', column_width=None), ColInfo(var='Q2', type=<ColInfoTypeEnum.default: 1>, column_label='Q2', column_align='right', column_width=None), ColInfo(var='Q3', type=<ColInfoTypeEnum.default: 1>, column_label='Q3', column_align='right', column_widt

#### Play neighborhood

In [196]:
# calculate mean & quartiles for risk attitude by play neighborhood
wealth_by_risk_playnhood = agents_last_round_df.group_by("risk_attitude", "play_neighborhood").agg(
    min=pl.col("scaled_points").min(), 
    max=pl.col("scaled_points").max(),
    median=pl.col("scaled_points").median(),
    mean=pl.col("scaled_points").mean(), 
    Q1=pl.col("scaled_points").quantile(0.25),
    Q2=pl.col("scaled_points").quantile(0.5),
    Q3=pl.col("scaled_points").quantile(0.75),
).sort("play_neighborhood", "risk_attitude")


In [216]:
custom_boxplot(wealth_by_risk_playnhood).facet(column=alt.Column("play_neighborhood", title="Play Neighborhood")).properties(title=wealthchart_title)

alt.FacetChart(...)

In [198]:
plot_mean_quartiles(wealth_by_risk_playnhood, 'play_neighborhood', 'Play Neighborhood')

alt.HConcatChart(...)

Click on items in the Play Neighborhood legend to change the chart opacity to focus on a particular group.

In [199]:
(wealth_by_risk_playnhood.with_columns(play_neighborhood=pl.lit("Play Neighborhood: ").add(pl.col("play_neighborhood").cast(pl.datatypes.String)))
    .rename({"risk_attitude": "Risk Attitude"})
    .style.tab_header(title="Cumulative Wealth by Risk Attitude and Play Neighborhood", subtitle="Wealth scaled by simulation length and play neighborhood")
     .tab_stub(rowname_col="Risk Attitude", groupname_col="play_neighborhood")
    .fmt_number(decimals=1)
    .fmt_number("Risk Attitude", decimals=0)
)

GT(_tbl_data=shape: (30, 9)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ Risk      ┆ play_neig ┆ min       ┆ max       ┆ … ┆ mean      ┆ Q1        ┆ Q2        ┆ Q3       │
│ Attitude  ┆ hborhood  ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│ ---       ┆ ---       ┆ f64       ┆ f64       ┆   ┆ f64       ┆ f64       ┆ f64       ┆ f64      │
│ i64       ┆ str       ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 0         ┆ Play Neig ┆ 0.0       ┆ 300.0     ┆ … ┆ 151.70060 ┆ 79.83871  ┆ 150.0     ┆ 220.1612 │
│           ┆ hborhood: ┆           ┆           ┆   ┆ 2         ┆           ┆           ┆ 9        │
│           ┆ 4         ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ 1         ┆ Play Neig ┆ 0.0       ┆ 300.0     ┆ … ┆ 148.56029 ┆ 88.709677 ┆ 149.51923 ┆ 191.1016 │
│           ┆ hborhood: ┆           ┆           ┆   ┆ 5         ┆           ┆ 1         ┆ 95       │
│           ┆ 4         ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ 2         ┆ Play Neig ┆ 0.0       ┆ 300.0     ┆ … ┆ 150.64837 ┆ 99.193548 ┆ 149.19354 ┆ 212.0967 │
│           ┆ hborhood: ┆           ┆           ┆   ┆ 5         ┆           ┆ 8         ┆ 74       │
│           ┆ 4         ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ 3         ┆ Play Neig ┆ 0.0       ┆ 300.0     ┆ … ┆ 147.68716 ┆ 101.81818 ┆ 129.50819 ┆ 186.5740 │
│           ┆ hborhood: ┆           ┆           ┆   ┆ 3         ┆ 2         ┆ 7         ┆ 74       │
│           ┆ 4         ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ 4         ┆ Play Neig ┆ 0.0       ┆ 300.0     ┆ … ┆ 145.39259 ┆ 102.08333 ┆ 125.51020 ┆ 175.0    │
│           ┆ hborhood: ┆           ┆           ┆   ┆ 2         ┆ 3         ┆ 4         ┆          │
│           ┆ 4         ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ …         ┆ …         ┆ …         ┆ …         ┆ … ┆ …         ┆ …         ┆ …         ┆ …        │
│ 5         ┆ Play Neig ┆ 82.598039 ┆ 266.52542 ┆ … ┆ 139.92863 ┆ 125.51020 ┆ 141.82098 ┆ 151.3440 │
│           ┆ hborhood: ┆           ┆ 4         ┆   ┆ 7         ┆ 4         ┆ 8         ┆ 86       │
│           ┆ 24        ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ 6         ┆ Play Neig ┆ 82.748538 ┆ 256.98924 ┆ … ┆ 143.76249 ┆ 134.95762 ┆ 145.96774 ┆ 154.1666 │
│           ┆ hborhood: ┆           ┆ 7         ┆   ┆ 6         ┆ 7         ┆ 2         ┆ 67       │
│           ┆ 24        ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ 7         ┆ Play Neig ┆ 90.607345 ┆ 244.75806 ┆ … ┆ 148.30579 ┆ 142.06989 ┆ 150.0     ┆ 156.9892 │
│           ┆ hborhood: ┆           ┆ 5         ┆   ┆ 4         ┆ 2         ┆           ┆ 47       │
│           ┆ 24        ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ 8         ┆ Play Neig ┆ 95.127119 ┆ 247.17741 ┆ … ┆ 151.94756 ┆ 145.96774 ┆ 153.22580 ┆ 158.6021 │
│           ┆ hborhood: ┆           ┆ 9         ┆   ┆ 2         ┆ 2         ┆ 6         ┆ 51       │
│           ┆ 24        ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ 9         ┆ Play Neig ┆ 113.30645 ┆ 189.91935 ┆ … ┆ 151.20844 ┆ 145.69892 ┆ 150.80645 ┆ 157.7956 │
│           ┆ hborhood: ┆ 2         ┆ 5         ┆   ┆ 8         ┆ 5         ┆ 2         ┆ 99       │
│           ┆ 24        ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
└───────────┴───────────┴───────────┴───────────┴───┴───────────┴───────────┴───────────┴──────────┘, _body=<great_tables._gt_data.Body object at 0x3d3ddbb00>, _boxhead=Boxhead([ColInfo(var='Risk Attitude', type=<ColInfoTypeEnum.stub: 

#### Observed neighborhood

In [200]:
# calculate mean & quartiles for risk attitude by play neighborhood
wealth_by_risk_obsvnhood = agents_last_round_df.group_by("risk_attitude", "observed_neighborhood").agg(
    min=pl.col("scaled_points").min(), 
    max=pl.col("scaled_points").max(),
    median=pl.col("scaled_points").median(),
    mean=pl.col("scaled_points").mean(), 
    Q1=pl.col("scaled_points").quantile(0.25),
    Q2=pl.col("scaled_points").quantile(0.5),
    Q3=pl.col("scaled_points").quantile(0.75),
).sort("observed_neighborhood", "risk_attitude")

In [217]:
custom_boxplot(wealth_by_risk_obsvnhood).facet(column=alt.Column("observed_neighborhood", title="Observed Neighborhood")).properties(title=wealthchart_title)

alt.FacetChart(...)

In [202]:
plot_mean_quartiles(wealth_by_risk_obsvnhood, 'observed_neighborhood', 'Observed Neighborhood')

alt.HConcatChart(...)

Click on items in the Observed Neighborhood legend to change the chart opacity to focus on a particular group.

In [203]:
(wealth_by_risk_obsvnhood.with_columns(observed_neighborhood=pl.lit("Observed Neighborhood: ").add(pl.col("observed_neighborhood").cast(pl.datatypes.String)))
    .rename({"risk_attitude": "Risk Attitude"})
    .style.tab_header(title="Cumulative Wealth by Risk Attitude and Observed Neighborhood", subtitle="Wealth scaled by simulation length and play neighborhood")
     .tab_stub(rowname_col="Risk Attitude", groupname_col="observed_neighborhood")
    .fmt_number(decimals=1)
    .fmt_number("Risk Attitude", decimals=0)
)

GT(_tbl_data=shape: (30, 9)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ Risk      ┆ observed_ ┆ min       ┆ max       ┆ … ┆ mean      ┆ Q1        ┆ Q2        ┆ Q3       │
│ Attitude  ┆ neighborh ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│ ---       ┆ ood       ┆ f64       ┆ f64       ┆   ┆ f64       ┆ f64       ┆ f64       ┆ f64      │
│ i64       ┆ ---       ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│           ┆ str       ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 0         ┆ Observed  ┆ 0.0       ┆ 300.0     ┆ … ┆ 153.05584 ┆ 125.54644 ┆ 150.0     ┆ 183.8709 │
│           ┆ Neighborh ┆           ┆           ┆   ┆ 3         ┆ 8         ┆           ┆ 68       │
│           ┆ ood: 4    ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ 1         ┆ Observed  ┆ 38.709677 ┆ 300.0     ┆ … ┆ 150.96171 ┆ 118.20652 ┆ 148.63505 ┆ 174.3279 │
│           ┆ Neighborh ┆           ┆           ┆   ┆ 3         ┆ 2         ┆ 7         ┆ 57       │
│           ┆ ood: 4    ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ 2         ┆ Observed  ┆ 38.709677 ┆ 300.0     ┆ … ┆ 154.9343  ┆ 124.57627 ┆ 149.59677 ┆ 183.0645 │
│           ┆ Neighborh ┆           ┆           ┆   ┆           ┆ 1         ┆ 4         ┆ 16       │
│           ┆ ood: 4    ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ 3         ┆ Observed  ┆ 67.741935 ┆ 300.0     ┆ … ┆ 152.79100 ┆ 124.19354 ┆ 145.29569 ┆ 168.5109 │
│           ┆ Neighborh ┆           ┆           ┆   ┆ 2         ┆ 8         ┆ 9         ┆ 29       │
│           ┆ ood: 4    ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ 4         ┆ Observed  ┆ 64.386792 ┆ 300.0     ┆ … ┆ 154.57354 ┆ 124.86559 ┆ 147.69230 ┆ 169.7580 │
│           ┆ Neighborh ┆           ┆           ┆   ┆ 4         ┆ 1         ┆ 8         ┆ 65       │
│           ┆ ood: 4    ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ …         ┆ …         ┆ …         ┆ …         ┆ … ┆ …         ┆ …         ┆ …         ┆ …        │
│ 5         ┆ Observed  ┆ 1.694915  ┆ 300.0     ┆ … ┆ 131.15903 ┆ 104.34782 ┆ 125.80645 ┆ 149.4623 │
│           ┆ Neighborh ┆           ┆           ┆   ┆ 2         ┆ 6         ┆ 2         ┆ 66       │
│           ┆ ood: 24   ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ 6         ┆ Observed  ┆ 43.220339 ┆ 298.5     ┆ … ┆ 136.96537 ┆ 111.82795 ┆ 138.20754 ┆ 152.4193 │
│           ┆ Neighborh ┆           ┆           ┆   ┆ 9         ┆ 7         ┆ 7         ┆ 55       │
│           ┆ ood: 24   ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ 7         ┆ Observed  ┆ 64.705882 ┆ 291.12903 ┆ … ┆ 145.00685 ┆ 131.45161 ┆ 149.19354 ┆ 160.3365 │
│           ┆ Neighborh ┆           ┆ 2         ┆   ┆           ┆ 3         ┆ 8         ┆ 38       │
│           ┆ ood: 24   ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ 8         ┆ Observed  ┆ 87.711864 ┆ 247.03389 ┆ … ┆ 151.55643 ┆ 141.93548 ┆ 151.34408 ┆ 162.5    │
│           ┆ Neighborh ┆           ┆ 8         ┆   ┆ 4         ┆ 4         ┆ 6         ┆          │
│           ┆ ood: 24   ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ 9         ┆ Observed  ┆ 96.774194 ┆ 203.22580 ┆ … ┆ 151.46619 ┆ 141.93548 ┆ 150.40322 ┆ 162.0967 │
│           ┆ Neighborh ┆           ┆ 6         ┆   ┆ 8         ┆ 4         ┆ 6         ┆ 74       │
│           ┆ ood: 24   ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
└───────────┴───────────┴───────────┴───────────┴───┴───────────┴───────────┴───────────┴──────────┘, _body=<great_tables._gt_data.Bod

#### Initial risk distribution

In [204]:
# calculate mean & quartiles for risk attitude by play neighborhood
wealth_by_risk_dist = agents_last_round_df.group_by("risk_attitude", "risk_distribution").agg(
    min=pl.col("scaled_points").min(), 
    max=pl.col("scaled_points").max(),
    median=pl.col("scaled_points").median(), 
    mean=pl.col("scaled_points").mean(), 
    Q1=pl.col("scaled_points").quantile(0.25),
    Q2=pl.col("scaled_points").quantile(0.5),
    Q3=pl.col("scaled_points").quantile(0.75),
).sort("risk_distribution", "risk_attitude")


In [218]:
custom_boxplot(wealth_by_risk_dist).facet(facet=alt.Facet("risk_distribution", title="Initial Risk Distribution"), columns=3
).properties(title=wealthchart_title)

alt.FacetChart(...)

In [206]:
plot_mean_quartiles(wealth_by_risk_dist, 'risk_distribution', 'Initial Risk Distribution')

alt.HConcatChart(...)

Click on items in the Distribution legend to change the chart opacity to focus on a particular group.

In [207]:
# output values as a nice table so we can reference them if needed
(wealth_by_risk_dist.with_columns(risk_distribution=pl.lit("Initial Risk Distribution: ").add(pl.col("risk_distribution")))
    .rename({"risk_attitude": "Risk Attitude"})
    .style.tab_header(title="Cumulative Wealth by Risk Attitude and Initial Risk Distribution", subtitle="Wealth scaled by simulation length and play neighborhood")
     .tab_stub(rowname_col="Risk Attitude", groupname_col="risk_distribution")
    .fmt_number(decimals=1)
    .fmt_number("Risk Attitude", decimals=0)
)

GT(_tbl_data=shape: (50, 9)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ Risk      ┆ risk_dist ┆ min       ┆ max       ┆ … ┆ mean      ┆ Q1        ┆ Q2        ┆ Q3       │
│ Attitude  ┆ ribution  ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│ ---       ┆ ---       ┆ f64       ┆ f64       ┆   ┆ f64       ┆ f64       ┆ f64       ┆ f64      │
│ i64       ┆ str       ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 0         ┆ Initial   ┆ 0.0       ┆ 300.0     ┆ … ┆ 152.12405 ┆ 124.77272 ┆ 150.0     ┆ 185.4838 │
│           ┆ Risk Dist ┆           ┆           ┆   ┆ 1         ┆ 7         ┆           ┆ 71       │
│           ┆ ribution: ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│           ┆ bim…      ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ 1         ┆ Initial   ┆ 0.0       ┆ 300.0     ┆ … ┆ 153.14757 ┆ 125.20833 ┆ 150.0     ┆ 185.4838 │
│           ┆ Risk Dist ┆           ┆           ┆   ┆ 4         ┆ 3         ┆           ┆ 71       │
│           ┆ ribution: ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│           ┆ bim…      ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ 2         ┆ Initial   ┆ 0.0       ┆ 300.0     ┆ … ┆ 153.28330 ┆ 125.40322 ┆ 150.0     ┆ 185.4838 │
│           ┆ Risk Dist ┆           ┆           ┆   ┆ 2         ┆ 6         ┆           ┆ 71       │
│           ┆ ribution: ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│           ┆ bim…      ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ 3         ┆ Initial   ┆ 0.0       ┆ 300.0     ┆ … ┆ 155.97132 ┆ 126.20967 ┆ 150.0     ┆ 185.0806 │
│           ┆ Risk Dist ┆           ┆           ┆   ┆ 2         ┆ 7         ┆           ┆ 45       │
│           ┆ ribution: ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│           ┆ bim…      ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ 4         ┆ Initial   ┆ 1.363636  ┆ 300.0     ┆ … ┆ 158.67531 ┆ 137.09677 ┆ 150.0     ┆ 184.6774 │
│           ┆ Risk Dist ┆           ┆           ┆   ┆           ┆ 4         ┆           ┆ 19       │
│           ┆ ribution: ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│           ┆ bim…      ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ …         ┆ …         ┆ …         ┆ …         ┆ … ┆ …         ┆ …         ┆ …         ┆ …        │
│ 5         ┆ Initial   ┆ 4.032258  ┆ 300.0     ┆ … ┆ 150.64528 ┆ 136.96236 ┆ 148.38709 ┆ 156.4814 │
│           ┆ Risk Dist ┆           ┆           ┆   ┆ 8         ┆ 6         ┆ 7         ┆ 81       │
│           ┆ ribution: ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│           ┆ uni…      ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ 6         ┆ Initial   ┆ 56.451613 ┆ 300.0     ┆ … ┆ 149.44544 ┆ 137.90322 ┆ 148.79032 ┆ 154.8387 │
│           ┆ Risk Dist ┆           ┆           ┆   ┆ 2         ┆ 6         ┆ 3         ┆ 1        │
│           ┆ ribution: ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│           ┆ uni…      ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ 7         ┆ Initial   ┆ 84.677419 ┆ 300.0     ┆ … ┆ 147.83073 ┆ 138.30645 ┆ 149.19354 ┆ 154.0322 │
│           ┆ Risk Dist ┆           ┆           ┆   ┆ 4         ┆ 2         ┆ 8         ┆ 58       │
│           ┆ ribution: ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│           ┆ uni…      ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ 8         ┆ Initial   ┆ 93.1451